In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.datasets import cifar10

# Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Custom generator to resize on-the-fly
class CIFAR10Generator(Sequence):
    def __init__(self, images, labels, batch_size=32):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x_resized = tf.image.resize(batch_x, [224, 224])
        batch_x_resized = preprocess_input(batch_x_resized)
        return batch_x_resized, batch_y

# แบ่ง validation set ชัดเจน เช่น 5000 ตัวอย่างสุดท้ายของ train เป็น validation
train_gen = CIFAR10Generator(x_train[:-5000], y_train[:-5000], batch_size=64)
val_gen = CIFAR10Generator(x_train[-5000:], y_train[-5000:], batch_size=64)
test_gen = CIFAR10Generator(x_test, y_test, batch_size=64)

# Load ResNet50 base model without top
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze base model ก่อน (ไม่ train)
base_model.trainable = False

# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train head only
model.fit(train_gen, epochs=5, validation_data=val_gen)

# Unfreeze base model เพื่อ fine-tune
base_model.trainable = True

# Compile ใหม่ด้วย learning rate ต่ำ
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune ทั้งโมเดล
model.fit(train_gen, epochs=5, validation_data=val_gen)

# Save model
model.save('cifar_tf.h5')

# Evaluate on test set
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc:.4f}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


704/704 ━━━━━━━━━━━━━━━━━━━━ 218s 288ms/step - accuracy: 0.8126 - loss: 0.5690 - val_accuracy: 0.8958 - val_loss: 0.2949
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 176s 250ms/step - accuracy: 0.9094 - loss: 0.2569 - val_accuracy: 0.8990 - val_loss: 0.2741
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 177s 251ms/step - accuracy: 0.9264 - loss: 0.2066 - val_accuracy: 0.9078 - val_loss: 0.2597
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 180s 256ms/step - accuracy: 0.9319 - loss: 0.1840 - val_accuracy: 0.9104 - val_loss: 0.2560
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 180s 256ms/step - accuracy: 0.9484 - loss: 0.1474 - val_accuracy: 0.9064 - val_loss: 0.2826
Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 536s 674ms/step - accuracy: 0.8304 - loss: 0.5470 - val_accuracy: 0.9300 - val_loss: 0.2142
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 445s 632ms/step - accuracy: 0.9777 - loss: 0.0786 - val_accuracy: 0.9348 - val_loss: 0.2010
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 448s 636ms/step - accuracy: 0.9979 - loss: 0.0212 - va

157/157 ━━━━━━━━━━━━━━━━━━━━ 40s 251ms/step - accuracy: 0.9376 - loss: 0.2280
Test Accuracy: 0.9377


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

# โหลดโมเดลที่ train แล้ว
model = tf.keras.models.load_model('/content/cifar_tf.h5')

# ชื่อคลาสใน CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

#โหลดและเตรียมภาพจากไฟล์
def load_and_predict(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # ทำนาย
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]

    print(f"Predicted class: {predicted_class} ({np.max(predictions) * 100:.2f}% confidence)")

#เปลี่ยน path เป็นรูปของคุณ
load_and_predict('/content/sample_data/test1.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted class: bird (100.00% confidence)
